In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import KFold
#%pip install lightgbm
import lightgbm as lgb

In [4]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import ExtraTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import SGDClassifier
%pip install deep-forest
from deepforest import CascadeForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [3]:
#%pip install scikit-learn=1.1
#注意，这个估计器目前还处于测试阶段: 预测和API可能会在没有任何弃用周期的情况下发生变化。要使用它，您需要显式导入enable_hist_gradient_boosting:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
#print(sklearn.__version__)
from sklearn.ensemble import HistGradientBoostingClassifier

In [17]:
url = "D:/1研究生学习资料/实验数据/实验数据/重做/重做-去重/Re去重/关键基因/RNA39作为未证实/final_train_RNA_data795.csv"
data = pd.read_csv(url)

### Load data set

In [18]:
print(data.shape)
#data.head(10)
data

(795, 7)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,y
0,1706.619722,-1.758654,0.204940,-8.581328,9.380000e-18,1.270000e-16,1
1,262.779843,1.104679,0.205058,5.387159,7.160000e-08,3.090000e-07,1
2,2881.943486,1.212687,0.392819,3.087136,2.020949e-03,4.306827e-03,1
3,228.182361,3.784003,0.949153,3.986718,6.700000e-05,1.857950e-04,1
4,6866.074347,2.430582,0.491265,4.947604,7.510000e-07,2.820000e-06,1
...,...,...,...,...,...,...,...
790,40.111376,3.860520,0.527058,7.324653,2.400000e-13,2.050000e-12,0
791,2528.928177,2.284136,0.247865,9.215224,3.110000e-20,5.160000e-19,0
792,71.091778,1.006783,0.253426,3.972691,7.110000e-05,1.959630e-04,0
793,300.568538,1.755329,0.335342,5.234437,1.650000e-07,6.800000e-07,0


In [19]:
data.iloc[:, -1].value_counts()

0    713
1     82
Name: y, dtype: int64

In [20]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print(f1) 
    rec = recall_score(y_test, y_predict)
    print(rec) 
    acc = accuracy_score(y_test, y_predict)
    print(acc)
    # 由于验证集中没有真实标签为0的样本，因此FP始终为0，则TP/(TP+FP)始终为100%
    pre = precision_score(y_test, y_predict)
    print(pre) 
    return f1,rec,acc,pre

    
#evaluate_results(y_test, y_predict)

In [26]:
mod_data = data.copy()
X = mod_data.loc[np.where(mod_data.iloc[:,-1].values == 1)[0]].iloc[:,:-1]#所有的正样本
y = mod_data.loc[np.where(mod_data.iloc[:,-1].values == 1)[0]].iloc[:,-1]#所有的正样本标签
kf = KFold(n_splits=5,shuffle=True,random_state=42)  # 初始化KFold
#dev_sum = 0
#result_sum = 0
f1_sum=0
rec_sum=0
acc_sum=0
pre_sum=0
max_f1=0
RANDOM_STATE = 42
for train_index , test_index in kf.split(X):  # 调用split方法切分数据
    mod_data['class_test'] = -1 #创建一个新列
    mod_data.loc[train_index,'class_test'] = 1 #训练集正样本的标签设置为1
    print('target variable:\n', mod_data.iloc[:,-1].value_counts()) #构建的新的数据集中正样本和未标记样本的比例情况
    print('train_index:%s , test_index: %s ' %(train_index,test_index))
    x_data = mod_data.iloc[:,:-2].values # just the X 
    y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
    y_positive = mod_data.iloc[:,-2].values # original class
    
    
    y_data = mod_data.iloc[:,-1]
    df_orig_positive  = mod_data.iloc[y_data.values == 1]#正标签所有行数据
    df_orig_unlabeled = mod_data.iloc[y_data.values != 1]#未标记标签所有行数据
    x_data_pos = df_orig_positive.iloc[:,:-2].values#正标签特征
    x_data_unl = df_orig_unlabeled.iloc[:,:-2].values#未标记标签特征
    
    len_pos = x_data_pos.shape[0] # size of positives  82
    len_unlabeled = x_data_unl.shape[0] #size of unlabeled 713
    learners_num = 128 #learners
    bootstrap_sample_size = len_pos # random bootstrap sample size
    
    #create a label set for each learning cycle
    #为每个学习周期创建一个标签集
    train_labels = np.zeros(shape=(len_pos + bootstrap_sample_size,)) #164
    #populate the first part of the set with the positive label, 
    #用正面标签填充第一部分，
    train_labels[:len_pos] = 1.0
    #place holder array for the number of times the datapoint is predicted
    #预测数据点的次数的占位符数组(袋外数据)
    n_oob = np.zeros(shape=(len_unlabeled,))
    #holds the results of the prediction of the data point
    #保存数据点的预测结果
    f_oob = np.zeros(shape=(len_unlabeled, 2))
    
    for i in range(learners_num):
        # Bootstrap resample
        bootstrap_sample = np.random.choice(np.arange(len_unlabeled), replace=True, size=bootstrap_sample_size)
        # Positive set + bootstrapped unlabeled set
        data_bootstrap = np.concatenate((x_data_pos,x_data_unl[bootstrap_sample, :]), axis=0)
        # Train model
        #model = lgb.LGBMClassifier()
        #model = DecisionTreeClassifier(max_depth=None, max_features=None,criterion='gini', class_weight='balanced')
        #model = RandomForestClassifier(random_state=RANDOM_STATE,n_estimators=300, criterion='entropy', max_depth=5)
        #model = KNeighborsClassifier(n_neighbors=4)
        model = MLPClassifier(random_state=RANDOM_STATE, verbose=0,hidden_layer_sizes=13)
        #model = make_pipeline(StandardScaler(),SVC(gamma='auto', probability=True,random_state=RANDOM_STATE, verbose=0))
        #model = make_pipeline(StandardScaler(),SGDClassifier(random_state=RANDOM_STATE,verbose=0, loss="log"))
        #model = LogisticRegression(random_state=RANDOM_STATE, penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)
        #model = CascadeForestClassifier(random_state=RANDOM_STATE, verbose=0,backend="sklearn")
        #model = AdaBoostClassifier(random_state=RANDOM_STATE)
        #model = GradientBoostingClassifier(random_state=RANDOM_STATE,verbose=0, n_estimators=100, max_depth=4)
        #model = LinearDiscriminantAnalysis()
        #model = ExtraTreesClassifier(random_state=RANDOM_STATE, verbose=0, n_estimators=100, criterion='gini', max_depth=3, min_samples_split=2, min_samples_leaf=2)
        #model = BaggingClassifier(random_state=RANDOM_STATE, verbose=0,base_estimator=None, n_estimators=100, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False)
        #model = BernoulliNB()
        #model = ExtraTreeClassifier(random_state=RANDOM_STATE)
        #model = GaussianProcessClassifier(random_state=RANDOM_STATE)
        #model = HistGradientBoostingClassifier(verbose=0,random_state=RANDOM_STATE, learning_rate=0.0001, max_depth=4)
        
        model.fit(data_bootstrap, train_labels)
        # Index for the out of the bag (oob) samples
        idx_oob = sorted(set(range(len_unlabeled)) - set(np.unique(bootstrap_sample)))
        # Transductive learning of oob samples
        #model.predict_proba(x)不同于model.predict()，它返回的预测值为获得所有结果的概率。（有多少个分类结果，每行就有多少个概率，对每个结果都有一个概率值，如0、1两分类就有两个概率）
        f_oob[idx_oob] += model.predict_proba(x_data_unl[idx_oob])
        #print("========================f_oob[idx_oob]========================================")
        #print(f_oob[idx_oob])
        n_oob[idx_oob] += 1
        if(i%10 == 0): print(f'learner {i}/{learners_num} completed')
        
    #某个样本x标记为正的概率P(y=1|x)等于样本被标记的概率P(s=1|x)除以正样本被标记的概率P(s=1|y=1)
    predicted = f_oob[:, 1]/n_oob
    
    df_orig_predicted = df_orig_unlabeled.copy()
    df_orig_predicted['pred'] = [1 if x > 0.5 else 0 for x in predicted]
    df_orig_positive.loc[:,'pred'] = 1
    df_outcome = pd.concat([df_orig_positive,df_orig_predicted])
    
    print(df_outcome)
    test=df_outcome.loc[(df_outcome["y"] == 1) & (df_outcome["class_test"] == -1)]
    print("========================test========================================")
    print(test)
    #选择y列为1，而class_test列为0的行，查看那几行中class_predict列为1的数量
    #dev_subsum=mod_data.loc[(mod_data["y"] == 1) &  (mod_data["class_test"] == 0)]
    #result=mod_data.loc[(mod_data["y"] == 1) &  (mod_data["class_test"] == 0) &  (mod_data["class_predict"] == 1)]
    predict_result=df_outcome.loc[(df_outcome["pred"] == 1) & (df_outcome["y"] == 0)]
    print("========================predict_result========================================")
    print(predict_result)
    
    print("========================结果========================================")
    results = evaluate_results(test["y"],test["pred"])
    f1_sum += results[0]
    rec_sum += results[1]
    acc_sum += results[2]
    pre_sum += results[3]
    
    #result_sum += result.shape[0]
    #print(result.shape[0])
    #dev_sum += dev_subsum.shape[0]
    #print(dev_subsum.shape[0])
    print("========================分界线======================================")
print(f1_sum / 5 ) 
print(rec_sum / 5 ) 
print(acc_sum / 5 ) 
print(pre_sum / 5 )
    
   

target variable:
 -1    730
 1     65
Name: class_test, dtype: int64
train_index:[ 1  2  3  5  6  7  8  9 11 13 14 15 16 17 19 20 21 23 24 25 26 27 29 32
 34 36 37 38 39 40 41 42 43 44 46 47 48 50 51 52 54 55 56 57 58 59 60 61
 62 63 64 65 66 67 70 71 72 73 74 76 77 78 79 80 81] , test_index: [ 0  4 10 12 18 22 28 30 31 33 35 45 49 53 68 69 75] 
learner 0/128 completed
learner 10/128 completed
learner 20/128 completed
learner 30/128 completed
learner 40/128 completed
learner 50/128 completed
learner 60/128 completed
learner 70/128 completed
learner 80/128 completed
learner 90/128 completed
learner 100/128 completed
learner 110/128 completed
learner 120/128 completed


D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


        baseMean  log2FoldChange     lfcSE      stat        pvalue  \
1     262.779843        1.104679  0.205058  5.387159  7.160000e-08   
2    2881.943486        1.212687  0.392819  3.087136  2.020949e-03   
3     228.182361        3.784003  0.949153  3.986718  6.700000e-05   
5    1895.491124        0.915327  0.294238  3.110842  1.865545e-03   
6     797.676718        1.485588  0.654231  2.270740  2.316272e-02   
..           ...             ...       ...       ...           ...   
790    40.111376        3.860520  0.527058  7.324653  2.400000e-13   
791  2528.928177        2.284136  0.247865  9.215224  3.110000e-20   
792    71.091778        1.006783  0.253426  3.972691  7.110000e-05   
793   300.568538        1.755329  0.335342  5.234437  1.650000e-07   
794   423.897324        4.035609  0.465272  8.673656  4.180000e-18   

             padj  y  class_test  pred  
1    3.090000e-07  1           1     1  
2    4.306827e-03  1           1     1  
3    1.857950e-04  1           1    

D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


        baseMean  log2FoldChange     lfcSE      stat        pvalue  \
0    1706.619722       -1.758654  0.204940 -8.581328  9.380000e-18   
1     262.779843        1.104679  0.205058  5.387159  7.160000e-08   
2    2881.943486        1.212687  0.392819  3.087136  2.020949e-03   
3     228.182361        3.784003  0.949153  3.986718  6.700000e-05   
4    6866.074347        2.430582  0.491265  4.947604  7.510000e-07   
..           ...             ...       ...       ...           ...   
790    40.111376        3.860520  0.527058  7.324653  2.400000e-13   
791  2528.928177        2.284136  0.247865  9.215224  3.110000e-20   
792    71.091778        1.006783  0.253426  3.972691  7.110000e-05   
793   300.568538        1.755329  0.335342  5.234437  1.650000e-07   
794   423.897324        4.035609  0.465272  8.673656  4.180000e-18   

             padj  y  class_test  pred  
0    1.270000e-16  1           1     1  
1    3.090000e-07  1           1     1  
2    4.306827e-03  1           1    

D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


        baseMean  log2FoldChange     lfcSE      stat        pvalue  \
0    1706.619722       -1.758654  0.204940 -8.581328  9.380000e-18   
1     262.779843        1.104679  0.205058  5.387159  7.160000e-08   
2    2881.943486        1.212687  0.392819  3.087136  2.020949e-03   
4    6866.074347        2.430582  0.491265  4.947604  7.510000e-07   
5    1895.491124        0.915327  0.294238  3.110842  1.865545e-03   
..           ...             ...       ...       ...           ...   
790    40.111376        3.860520  0.527058  7.324653  2.400000e-13   
791  2528.928177        2.284136  0.247865  9.215224  3.110000e-20   
792    71.091778        1.006783  0.253426  3.972691  7.110000e-05   
793   300.568538        1.755329  0.335342  5.234437  1.650000e-07   
794   423.897324        4.035609  0.465272  8.673656  4.180000e-18   

             padj  y  class_test  pred  
0    1.270000e-16  1           1     1  
1    3.090000e-07  1           1     1  
2    4.306827e-03  1           1    

D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


        baseMean  log2FoldChange     lfcSE      stat        pvalue  \
0    1706.619722       -1.758654  0.204940 -8.581328  9.380000e-18   
1     262.779843        1.104679  0.205058  5.387159  7.160000e-08   
2    2881.943486        1.212687  0.392819  3.087136  2.020949e-03   
3     228.182361        3.784003  0.949153  3.986718  6.700000e-05   
4    6866.074347        2.430582  0.491265  4.947604  7.510000e-07   
..           ...             ...       ...       ...           ...   
790    40.111376        3.860520  0.527058  7.324653  2.400000e-13   
791  2528.928177        2.284136  0.247865  9.215224  3.110000e-20   
792    71.091778        1.006783  0.253426  3.972691  7.110000e-05   
793   300.568538        1.755329  0.335342  5.234437  1.650000e-07   
794   423.897324        4.035609  0.465272  8.673656  4.180000e-18   

             padj  y  class_test  pred  
0    1.270000e-16  1           1     1  
1    3.090000e-07  1           1     1  
2    4.306827e-03  1           1    

D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\softinstall\anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
#df_outcome.to_csv('D:/1研究生学习资料/实验数据/实验数据/重做/重做-去重/Re去重/关键基因/RNA39作为未证实/MLP_final_predict.csv',index=False)